# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [7]:
def select_relevant_links(url):
    response = openai.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        reasoning={"effort":"low"},
        text={"verbosity": "low"}
    )
    result = response.output_text
    links = json.loads(result)
    print(f"Total token usage:{response.usage.total_tokens}")
    return links
    

In [24]:
# gpt-5-nano
select_relevant_links("https://edwarddonner.com")

Total token usage:4476


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'brand/affiliation page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [41]:
# gpt-5-nano min efforts
select_relevant_links("https://edwarddonner.com")

Total token usage:1120


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'live events or blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'ai/tech blog post',
   'url': 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/'},
  {'type': 'ai/tech blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'ai/tech blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'connect-four game',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'outsmart page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'news a

In [8]:
# gpt-5-nano low efforts
select_relevant_links("https://edwarddonner.com")

Total token usage:1331


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [39]:
# gpt-5-nano medium effort
select_relevant_links("https://edwarddonner.com")

Total token usage:2997


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [32]:
# gpt-4.1-mini
select_relevant_links("https://edwarddonner.com")

Total token usage:705


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [34]:
# gpt-4.1-nano
select_relevant_links("https://edwarddonner.com")

Total token usage:1060


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'curriculum or expertise',
   'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'projects or products',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'projects or initiatives',
   'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'posts or blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'news or updates',
   'url': 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/'},
  {'type': 'news or updates',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'news or updates',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'learning or courses',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company/product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'blog / posts', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 17 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
5 days ago
•
736k
•
866
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
6 days ago
•
23.9k
•
527
tencent/HY-MT1.5-1.8B
Updated
12 days ago
•
10.7k
•
735
nvidia/nemotron-speech-streaming-en-0.6b
Updated
7 days ago
•
2.69k
•
325
LiquidAI/LFM2.5-1.2B-Instruct
Updated
4 days ago
•
12.8k
•
296
Browse 2M+ models
Spaces
Running
Featured
4.15k
Wan2.2 Animate
👁
4.15k
Wan2.2 Animate
Running
on
Zero
Featured
567
Qwen Image Multiple Angles 3D Camera
🎥
567
Adjust camera angles in images using 3D controls or sliders


In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n5 days ago\n•\n736k\n•\n866\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n6 days ago\n•\n23.9k\n•\n527\ntencent/HY-MT1.5-1.8B\nUpdated\n12 days ago\n•\n10.7k\n•\n735\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n7 days ago\n•\n2.69k\n•\n325\nLiquidAI/LFM2.5-1.2B-Instruct\nUpdated\n4 days ago\n•\n12.8k\n•\n297\nBrowse 2M+ models\nSpaces\nRunning\nF

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## Who We Are  
Hugging Face is the AI community building the future of machine learning. We provide an open platform where researchers, developers, and enterprises collaborate to advance AI by sharing models, datasets, and applications across multiple modalities including text, image, video, audio, and 3D.

---

## Our Platform

- **Models:** Access and contribute to a vast repository of over 2 million machine learning models, featuring the latest innovations updated regularly by a vibrant community.  
- **Datasets:** Discover and share from over 500,000 datasets tailored for diverse machine learning projects.  
- **Spaces:** Host and explore engaging AI-powered applications and demos built by the community. Over 1 million applications for exploration.  
- **Community Collaboration:** Connect with a global AI community to create, discover, and collaborate on cutting-edge machine learning projects and build your personal ML portfolio.  

---

## Enterprise Solutions

Hugging Face offers an enterprise-grade AI platform tailored for secure, scalable organizational use:  

- **Team Plans:** Starting at $20 per user/month, ideal for smaller teams and organizations.  
- **Enterprise Plans:** Customized contracts providing advanced features such as:  
  - Single Sign-On (SSO) integration for secure access  
  - Regional data location management and auditing  
  - Comprehensive audit logs for monitoring all actions  
  - Dedicated support and advanced resource controls  

Our platform empowers your organization to deploy AI projects efficiently with enterprise-grade security and collaboration capabilities.

---

## Company Culture

- **Open and Collaborative:** Hugging Face thrives on openness, with an emphasis on sharing knowledge openly and building together.  
- **Innovative and Community-Driven:** The beating heart of Hugging Face is its active community contributing to open-source machine learning advancements.  
- **Supportive Growth:** We foster an environment where individuals can share their work, develop skills, and grow their machine learning profiles.  

---

## For Our Customers

- Researchers and Developers: Access the most extensive collection of models, datasets, and tools to accelerate AI research and production.  
- Enterprises: Benefit from secure, scalable AI infrastructure with enterprise-grade features for building and deploying machine learning solutions.  
- AI Enthusiasts and Students: Build and showcase your AI projects while engaging with a vibrant community.

---

## Careers & Opportunities  

Join Hugging Face to be part of a passionate, innovative team at the forefront of AI:  

- Contribute to open-source projects used worldwide  
- Work alongside leading experts in machine learning  
- Grow your career in a mission-driven, community-focused environment  
- Access opportunities spanning research, engineering, product, and more  

For the latest openings and application details, visit our website’s careers section.

---

## Why Choose Hugging Face?

- Trusted by millions worldwide with 2M+ models and 1M+ applications  
- Home to diverse modalities from NLP to image and 3D AI  
- Powerful yet user-friendly tools to accelerate machine learning workflows  
- Robust enterprise-grade solutions ensuring security, compliance, and support  

---

### Connect with Us  
Explore, collaborate, and build the future of AI together.  
Website: [huggingface.co](https://huggingface.co)  
Sign Up and start accelerating your machine learning journey today!

---

Hugging Face — The AI community building the future.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>